In [0]:
from skimage import feature
import numpy as np
import pandas as pd
def extract_lbp(image, bb):
  eps=1e-7
  # own lbp extractor 

  #obtain bounding box
  #bb = tracker.getLargestFaceBoundingBox(image, skipMulti=False)

  #extract bb parameters
  x = bb.left()
  y = bb.top()
  w = bb.right() - x
  h = bb.bottom() - y

  #crop and grayscale the image 
  cropped = image[y:y+h,x:x+w,:]
  gray = cv2.cvtColor(cropped,cv2.COLOR_BGR2GRAY)

  #extract local binary pattern 
  lbp = feature.local_binary_pattern(gray, 24, 4, method="uniform")

  #obtain histogram for each region
  y_step = lbp.shape[0]//4
  x_step = lbp.shape[1]//4


  #computer resulting histograms
  hists =  []

  for l in range(4):
    for p in range(4):
        section = gray[ l*y_step: (l+1)*y_step, p*x_step: (p+1)*x_step]

        if l == 3 and p == 3:
          section = gray[ l*y_step: , p*x_step:]
        elif l == 3 and p != 3:
          section = gray[ l*y_step: , p*x_step: (p+1)*x_step]
        elif l != 3 and p == 3:
          section = gray[ l*y_step: (l+1)*y_step, p*x_step: ]
        else:
          section = gray[ l*y_step: (l+1)*y_step, p*x_step: (p+1)*x_step]

        (hist, _) = np.histogram(section.ravel(), bins=np.arange(0, 24 + 3))
        # normalize the histogram
        hist = hist.astype("float")
        hist /= (hist.sum() + eps)

        hists.append(hist)

  return np.array(hists)

In [0]:
def ravel_lbps(lbps):
  new_lbps = []
  for lbp in lbps:
    new_lbp = np.array(lbp).reshape(12, 416)
    new_lbps.append(new_lbp)

  return new_lbps